In [2]:
from pyspark.sql import SparkSession

# Crear la sesión de Spark
spark = SparkSession.builder.appName("Fabric_Lakehouse_Read_silver").getOrCreate()

# Definir la ruta del archivo Parquet
parquet_path = "Files/silver/datos_silver.parquet"

# Leer el archivo Parquet
df = spark.read.format("parquet").load(parquet_path)

# Mostrar los datos leídos
df.show()

# Contar las filas del DataFrame
print("Número de filas:", df.count())


StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 4, Finished, Available, Finished)

+--------+------------------+-------------------+-----------+-----------+-------------------+
|Variable|   nombre_variable|Measurement_Numeric|Measurement|Mapped_Unit|          Timestamp|
+--------+------------------+-------------------+-----------+-----------+-------------------+
|      Sm|Wind speed average|                3.4|       3.4M|        m/s|2024-11-02 15:30:25|
|      Sx|Wind speed maximum|                3.4|       3.4M|        m/s|2024-11-02 15:31:34|
|      Sx|Wind speed maximum|                3.4|       3.4M|        m/s|2024-11-02 15:31:35|
|      Sm|Wind speed average|                3.4|       3.4M|        m/s|2024-11-02 15:31:12|
|      Sm|Wind speed average|                3.4|       3.4M|        m/s|2024-11-02 15:31:14|
|      Sm|Wind speed average|                3.4|       3.4M|        m/s|2024-11-02 15:30:05|
|      Sm|Wind speed average|                3.4|       3.4M|        m/s|2024-11-02 15:31:20|
|      Sm|Wind speed average|                3.4|       3.4M

In [3]:
df= df.select("nombre_variable", "Measurement_Numeric","Mapped_Unit","Timestamp")
df.show()

StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 5, Finished, Available, Finished)

+------------------+-------------------+-----------+-------------------+
|   nombre_variable|Measurement_Numeric|Mapped_Unit|          Timestamp|
+------------------+-------------------+-----------+-------------------+
|Wind speed average|                3.4|        m/s|2024-11-02 15:30:25|
|Wind speed maximum|                3.4|        m/s|2024-11-02 15:31:34|
|Wind speed maximum|                3.4|        m/s|2024-11-02 15:31:35|
|Wind speed average|                3.4|        m/s|2024-11-02 15:31:12|
|Wind speed average|                3.4|        m/s|2024-11-02 15:31:14|
|Wind speed average|                3.4|        m/s|2024-11-02 15:30:05|
|Wind speed average|                3.4|        m/s|2024-11-02 15:31:20|
|Wind speed average|                3.4|        m/s|2024-11-02 15:31:21|
|Wind speed average|                3.4|        m/s|2024-11-02 15:30:24|
|Wind speed average|                3.4|        m/s|2024-11-02 15:31:26|
|Wind speed average|                3.4|        m/s

In [4]:
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, FloatType, TimestampType

# Convertir las columnas al tipo de dato adecuado
df_formatted = df \
    .withColumn("nombre_variable", F.col("nombre_variable").cast(StringType())) \
    .withColumn("Measurement_Numeric", F.col("Measurement_Numeric").cast(FloatType())) \
    .withColumn("Mapped_Unit", F.col("Mapped_Unit").cast(StringType())) \
    .withColumn("Timestamp", F.col("Timestamp").cast(TimestampType()))

# Mostrar el esquema para verificar los tipos de datos
df_formatted.printSchema()

# Mostrar los datos
df_formatted.show(truncate=False)


StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 6, Finished, Available, Finished)

root
 |-- nombre_variable: string (nullable = true)
 |-- Measurement_Numeric: float (nullable = true)
 |-- Mapped_Unit: string (nullable = true)
 |-- Timestamp: timestamp (nullable = true)

+------------------+-------------------+-----------+-------------------+
|nombre_variable   |Measurement_Numeric|Mapped_Unit|Timestamp          |
+------------------+-------------------+-----------+-------------------+
|Wind speed average|3.4                |m/s        |2024-11-02 15:30:25|
|Wind speed maximum|3.4                |m/s        |2024-11-02 15:31:34|
|Wind speed maximum|3.4                |m/s        |2024-11-02 15:31:35|
|Wind speed average|3.4                |m/s        |2024-11-02 15:31:12|
|Wind speed average|3.4                |m/s        |2024-11-02 15:31:14|
|Wind speed average|3.4                |m/s        |2024-11-02 15:30:05|
|Wind speed average|3.4                |m/s        |2024-11-02 15:31:20|
|Wind speed average|3.4                |m/s        |2024-11-02 15:31:21|
|Wind s

In [5]:
import shutil
import os

# Ruta a la carpeta y archivo Parquet en la carpeta 'gold'
path = "Files/gold"  # Ruta a la carpeta 'gold'
parquet_file = f"{path}/datos_gold.parquet"  # Archivo Parquet

# Verificar si la carpeta 'silver' existe, si no, crearla
if not os.path.exists(path):
    os.makedirs(path)

# Si el archivo parquet ya existe, eliminarlo
if os.path.exists(parquet_file):
    os.remove(parquet_file)  # Cambié shutil.rmtree por os.remove para eliminar un archivo

# Guardar el DataFrame en formato Parquet en la carpeta 'gold'
df_formatted.write.mode("overwrite").parquet(parquet_file)

print(f"Archivo Parquet guardado en {parquet_file}")


StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 7, Finished, Available, Finished)

Archivo Parquet guardado en Files/gold/datos_gold.parquet


In [6]:
# Guardar en una tabla SQL o Delta Lake
df_formatted.write.mode("overwrite").saveAsTable("tabla_gold")

print("Datos guardados en la tabla: tabla_gold")


StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 8, Finished, Available, Finished)

Datos guardados en la tabla: tabla_gold


In [7]:
# Leer la tabla guardada en Spark
df_gold = spark.table("tabla_gold")

# Mostrar los datos de la tabla
df_gold.show()


StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 9, Finished, Available, Finished)

+------------------+-------------------+-----------+-------------------+
|   nombre_variable|Measurement_Numeric|Mapped_Unit|          Timestamp|
+------------------+-------------------+-----------+-------------------+
| Relative humidity|               80.7|        %RH|2024-11-02 18:57:25|
|     Hail duration|               20.0|          s|2024-11-02 18:57:25|
|    Supply voltage|               24.3|          V|2024-11-02 18:57:25|
|3.5 V ref. voltage|              3.643|          V|2024-11-02 18:57:25|
|      Air pressure|              883.4|        hPa|2024-11-02 18:57:25|
|    Hail intensity|                0.0|  hits/cm²h|2024-11-02 18:57:25|
|   Air temperature|                9.0|         °C|2024-11-02 18:57:25|
| Rain accumulation|               1.07|         mm|2024-11-02 18:57:25|
|     Rain duration|             1800.0|          s|2024-11-02 18:57:25|
|    Rain intensity|                0.0|       mm/h|2024-11-02 18:57:25|
| Hail accumulation|                0.0|   hits/cm²

In [8]:
# crea Wind_table
# Filtrar el DataFrame usando el método isin
df_Wind = df_gold.filter(
    df_gold['nombre_variable'].isin(['Wind speed maximum', 'Wind speed minimum', 'Wind speed average', 'Wind direction maximum', 'Wind direction average','Wind direction minimum'])
)

# Mostrar el DataFrame filtrado
df_Wind.show()

StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 10, Finished, Available, Finished)

+--------------------+-------------------+-----------+-------------------+
|     nombre_variable|Measurement_Numeric|Mapped_Unit|          Timestamp|
+--------------------+-------------------+-----------+-------------------+
|Wind direction av...|              165.0|        deg|2024-11-02 15:30:34|
|Wind direction av...|              169.0|        deg|2024-11-02 15:31:40|
|Wind direction av...|              312.0|        deg|2024-11-02 18:57:23|
|Wind direction av...|              172.0|        deg|2024-11-02 15:31:04|
|Wind direction av...|              172.0|        deg|2024-11-02 15:31:05|
|Wind direction av...|              170.0|        deg|2024-11-02 15:30:44|
|Wind direction av...|              156.0|        deg|2024-11-02 15:30:13|
|Wind direction av...|              156.0|        deg|2024-11-02 15:30:14|
|Wind direction av...|              168.0|        deg|2024-11-02 15:31:25|
|Wind direction av...|              169.0|        deg|2024-11-02 15:31:16|
|Wind direction av...|   

In [9]:
# Guardar en una tabla SQL o Delta Lake
df_Wind.write.mode("overwrite").saveAsTable("wind_table")

print("Datos guardados en la tabla: Wind_table")

StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 11, Finished, Available, Finished)

Datos guardados en la tabla: Wind_table


In [10]:
# crea  Tabla de Presión y Temperatura 'Pa', 'Ta', 'Tp' 
# Filtrar el DataFrame usando el método isin
df_temperature_pressure = df_gold.filter(
    df_gold['nombre_variable'].isin(['Air pressure', 'Air temperature', 'Internal temperatur'])
)

# Mostrar el DataFrame filtrado
df_temperature_pressure.show()

StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 12, Finished, Available, Finished)

+---------------+-------------------+-----------+-------------------+
|nombre_variable|Measurement_Numeric|Mapped_Unit|          Timestamp|
+---------------+-------------------+-----------+-------------------+
|   Air pressure|              883.4|        hPa|2024-11-02 18:57:25|
|Air temperature|                9.0|         °C|2024-11-02 18:57:25|
|   Air pressure|              882.8|        hPa|2024-11-02 15:31:25|
|Air temperature|                6.1|         °C|2024-11-02 15:31:25|
|   Air pressure|              882.8|        hPa|2024-11-02 15:30:25|
|Air temperature|                6.2|         °C|2024-11-02 15:30:25|
|   Air pressure|              882.8|        hPa|2024-11-02 15:31:16|
|   Air pressure|              882.7|        hPa|2024-11-02 15:30:36|
|   Air pressure|              883.3|        hPa|2024-11-02 18:57:26|
|   Air pressure|              882.8|        hPa|2024-11-02 15:30:16|
|   Air pressure|              882.8|        hPa|2024-11-02 15:31:06|
|   Air pressure|   

In [11]:
# Guardar en una tabla SQL o Delta Lake
df_temperature_pressure.write.mode("overwrite").saveAsTable("temperature_pressure_table")

print("Datos guardados en la tabla: temperature_pressure_table")

StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 13, Finished, Available, Finished)

Datos guardados en la tabla: temperature_pressure_table


In [12]:
# crea  Tabla de 'Ua' 'Relative humidity'
# Filtrar el DataFrame usando el método isin
df_humidity = df_gold.filter(
    df_gold['nombre_variable'].isin(['Relative humidity'])
)

# Mostrar el DataFrame filtrado
df_humidity.show()

StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 14, Finished, Available, Finished)

+-----------------+-------------------+-----------+-------------------+
|  nombre_variable|Measurement_Numeric|Mapped_Unit|          Timestamp|
+-----------------+-------------------+-----------+-------------------+
|Relative humidity|               80.7|        %RH|2024-11-02 18:57:25|
|Relative humidity|              100.0|        %RH|2024-11-02 15:31:25|
|Relative humidity|              100.0|        %RH|2024-11-02 15:30:25|
|Relative humidity|              100.0|        %RH|2024-11-02 15:31:36|
|Relative humidity|              100.0|        %RH|2024-11-02 15:31:06|
|Relative humidity|              100.0|        %RH|2024-11-02 15:30:16|
|Relative humidity|               80.8|        %RH|2024-11-02 18:57:26|
|Relative humidity|              100.0|        %RH|2024-11-02 15:30:36|
|Relative humidity|              100.0|        %RH|2024-11-02 15:31:16|
|Relative humidity|              100.0|        %RH|2024-11-02 15:31:26|
|Relative humidity|              100.0|        %RH|2024-11-02 15

In [13]:
# Guardar en una tabla SQL o Delta Lake
df_humidity.write.mode("overwrite").saveAsTable("Relative_humidity_table")

print("Datos guardados en la tabla: Relative_humidity_table")

StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 15, Finished, Available, Finished)

Datos guardados en la tabla: Relative_humidity_table


In [14]:
# crea  Tabla de Rain
# Filtrar el DataFrame usando el método isin
df_rain = df_gold.filter(
    df_gold['nombre_variable'].isin(['Rain accumulation', 'Rain duration', 'Rain intensity', 'Rain peak intensity'])
)


# Mostrar el DataFrame filtrado
df_rain.show()


StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 16, Finished, Available, Finished)

+-----------------+-------------------+-----------+-------------------+
|  nombre_variable|Measurement_Numeric|Mapped_Unit|          Timestamp|
+-----------------+-------------------+-----------+-------------------+
|Rain accumulation|               1.07|         mm|2024-11-02 18:57:25|
|    Rain duration|             1800.0|          s|2024-11-02 18:57:25|
|   Rain intensity|                0.0|       mm/h|2024-11-02 18:57:25|
|   Rain intensity|                0.0|       mm/h|2024-11-02 15:31:25|
|Rain accumulation|               1.07|         mm|2024-11-02 15:31:25|
|    Rain duration|             1800.0|          s|2024-11-02 15:31:25|
|Rain accumulation|               1.07|         mm|2024-11-02 15:30:25|
|    Rain duration|             1800.0|          s|2024-11-02 15:30:25|
|   Rain intensity|                0.0|       mm/h|2024-11-02 15:30:25|
+-----------------+-------------------+-----------+-------------------+



In [15]:
# Guardar en una tabla SQL o Delta Lake
df_rain.write.mode("overwrite").saveAsTable("Rain_table")

print("Datos guardados en la tabla: Rain_table")

StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 17, Finished, Available, Finished)

Datos guardados en la tabla: Rain_table


In [16]:
# crea  Tabla de Hail
# Filtrar el DataFrame usando el método isin
df_hail = df_gold.filter(
    df_gold['nombre_variable'].isin(['Hail accumulation', 'Hail duration', 'Hail intensity', 'Hail peak intensity'])
)


# Mostrar el DataFrame filtrado
df_hail.show()


StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 18, Finished, Available, Finished)

+-----------------+-------------------+-----------+-------------------+
|  nombre_variable|Measurement_Numeric|Mapped_Unit|          Timestamp|
+-----------------+-------------------+-----------+-------------------+
|    Hail duration|               20.0|          s|2024-11-02 18:57:25|
|   Hail intensity|                0.0|  hits/cm²h|2024-11-02 18:57:25|
|Hail accumulation|                0.0|   hits/cm²|2024-11-02 18:57:25|
|    Hail duration|               20.0|          s|2024-11-02 15:31:25|
|   Hail intensity|                0.0|  hits/cm²h|2024-11-02 15:31:25|
|Hail accumulation|                0.0|   hits/cm²|2024-11-02 15:31:25|
|Hail accumulation|                0.0|   hits/cm²|2024-11-02 15:30:25|
|    Hail duration|               20.0|          s|2024-11-02 15:30:25|
|   Hail intensity|                0.0|  hits/cm²h|2024-11-02 15:30:25|
+-----------------+-------------------+-----------+-------------------+



In [17]:
# Guardar en una tabla SQL o Delta Lake
df_hail.write.mode("overwrite").saveAsTable("Hail_table")

print("Datos guardados en la tabla: Hail_table")

StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 19, Finished, Available, Finished)

Datos guardados en la tabla: Hail_table


In [18]:
# crea  Tabla de Heating
# Filtrar el DataFrame usando el método isin
df_Heating = df_gold.filter(
    df_gold['nombre_variable'].isin(['Heating temperature', 'Heating voltage'])
)


# Mostrar el DataFrame filtrado
df_Heating.show()


StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 20, Finished, Available, Finished)

+-------------------+-------------------+-----------+-------------------+
|    nombre_variable|Measurement_Numeric|Mapped_Unit|          Timestamp|
+-------------------+-------------------+-----------+-------------------+
|    Heating voltage|               22.6|          V|2024-11-02 15:30:23|
|    Heating voltage|               23.9|          V|2024-11-02 18:57:23|
|    Heating voltage|               23.9|          V|2024-11-02 15:31:23|
|Heating temperature|               15.0|         °C|2024-11-02 18:57:23|
|Heating temperature|               14.6|         °C|2024-11-02 15:30:23|
|Heating temperature|               15.0|         °C|2024-11-02 15:31:23|
+-------------------+-------------------+-----------+-------------------+



In [19]:
# Guardar en una tabla SQL o Delta Lake
df_Heating.write.mode("overwrite").saveAsTable("Heating_table")

print("Datos guardados en la tabla: Heating_table")

StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 21, Finished, Available, Finished)

Datos guardados en la tabla: Heating_table


In [20]:
# crea  Tabla de voltage
# Filtrar el DataFrame usando el método isin
df_voltage = df_gold.filter(
    df_gold['nombre_variable'].isin(['Supply voltage', '3.5 V ref. voltage'])
)


# Mostrar el DataFrame filtrado
df_voltage.show()

StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 22, Finished, Available, Finished)

+------------------+-------------------+-----------+-------------------+
|   nombre_variable|Measurement_Numeric|Mapped_Unit|          Timestamp|
+------------------+-------------------+-----------+-------------------+
|    Supply voltage|               24.3|          V|2024-11-02 18:57:25|
|3.5 V ref. voltage|              3.643|          V|2024-11-02 18:57:25|
|3.5 V ref. voltage|              3.643|          V|2024-11-02 15:31:25|
|    Supply voltage|               24.4|          V|2024-11-02 15:31:25|
|    Supply voltage|               23.3|          V|2024-11-02 15:30:25|
|3.5 V ref. voltage|              3.643|          V|2024-11-02 15:30:25|
|    Supply voltage|               24.4|          V|2024-11-02 15:31:23|
|3.5 V ref. voltage|              3.643|          V|2024-11-02 15:30:23|
|    Supply voltage|               23.3|          V|2024-11-02 15:30:23|
|    Supply voltage|               24.4|          V|2024-11-02 18:57:23|
|3.5 V ref. voltage|              3.636|          V

In [21]:
# Guardar en una tabla SQL o Delta Lake
df_Heating.write.mode("overwrite").saveAsTable("voltage_table")

print("Datos guardados en la tabla: voltage_table")

StatementMeta(, 29d9ba68-8ed9-4640-b1e1-cdbc6d7882ae, 23, Finished, Available, Finished)

Datos guardados en la tabla: voltage_table
